## Bayesian methods of hyperparameter optimization
* Notebook completed by Adam Lang, Springboard Data Science Fellow
* Date: 5/16/2024

In addition to the random search and the grid search methods for selecting optimal hyperparameters, we can use Bayesian methods of probabilities to select the optimal hyperparameters for an algorithm.

In this case study, we will be using the BayesianOptimization library to perform hyperparmater tuning. This library has very good documentation which you can find here: https://github.com/fmfn/BayesianOptimization

You will need to install the Bayesian optimization module. Running a cell with an exclamation point in the beginning of the command will run it as a shell command — please do this to install this module from our notebook in the cell below.

In [1]:
!pip install bayesian-optimization

In [3]:
## install catboost
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 2.2 MB/s eta 0:00:00


In [4]:
## import libraries
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

## bayesian imports
import lightgbm
from bayes_opt import BayesianOptimization
from catboost import CatBoostClassifier, cv, Pool

In [6]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# import os
import os
os.listdir()

['.config', 'drive', 'sample_data']

## How does Bayesian optimization work?

Bayesian optimization works by constructing a posterior distribution of functions (Gaussian process) that best describes the function you want to optimize. As the number of observations grows, the posterior distribution improves, and the algorithm becomes more certain of which regions in parameter space are worth exploring and which are not, as seen in the picture below.

<img src="https://github.com/fmfn/BayesianOptimization/blob/master/examples/bo_example.png?raw=true" />
As you iterate over and over, the algorithm balances its needs of exploration and exploitation while taking into account what it knows about the target function. At each step, a Gaussian Process is fitted to the known samples (points previously explored), and the posterior distribution, combined with an exploration strategy (such as UCB — aka Upper Confidence Bound), or EI (Expected Improvement). This process is used to determine the next point that should be explored (see the gif below).
<img src="https://github.com/fmfn/BayesianOptimization/raw/master/examples/bayesian_optimization.gif" />

## Let's look at a simple example

The first step is to create an optimizer. It uses two items:
* function to optimize
* bounds of parameters

The function is the procedure that counts metrics of our model quality. The important thing is that our optimization will maximize the value on function. **Smaller metrics are best.** Hint: don't forget to use negative metric values.

Here we define our simple function we want to optimize.

In [8]:
## simple function to optimize
def simple_func(a, b):
    return a + b

Now, we define our bounds of the parameters to optimize, within the Bayesian optimizer.

In [9]:
# bounds of params to optimize
optimizer = BayesianOptimization(
    simple_func,
    {'a': (1, 3),
    'b': (4, 7)})

These are the main parameters of this function:

* **n_iter:** This is how many steps of Bayesian optimization you want to perform. The more steps, the more likely you are to find a good maximum.

* **init_points:** This is how many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.

Let's run an example where we use the optimizer to find the best values to maximize the target value for a and b given the inputs of 3 and 2.

In [10]:
optimizer.maximize(3,2)

|   iter    |  target   |     a     |     b     |
-------------------------------------------------
| 1         | 7.669     | 1.815     | 5.854     |
| 2         | 7.923     | 1.201     | 6.722     |
| 3         | 7.415     | 1.475     | 5.941     |
| 4         | 6.505     | 2.434     | 4.072     |
| 5         | 8.821     | 1.915     | 6.906     |


Great, now let's print the best parameters and the associated maximized target.

In [11]:
# print best params and max target
print(optimizer.max['params']);optimizer.max['target']

{'a': 1.9149664699334112, 'b': 6.9064312224743665}


8.821397692407778

## Test it on real data using the Light GBM

The dataset we will be working with is the famous **flight departures dataset.**
* Our modeling goal will be to **predict if a flight departure is going to be delayed by 15 minutes based on the other attributes in our dataset.**
* As part of this modeling exercise, we will use Bayesian hyperparameter optimization to identify the best parameters for our model.

**<font color='teal'> You can load the zipped csv files just as you would regular csv files using Pandas read_csv. In the next cell load the train and test data into two seperate dataframes. </font>**


In [13]:
## need path in google drive to load these
data_train_path = '/content/drive/MyDrive/Springboard_Data_Science/Model Optimization/Bayesian_Optimization_Case_Study/Bayesian Optimization Case Study/flight_delays_train.csv.zip'
data_test_path = '/content/drive/MyDrive/Springboard_Data_Science/Model Optimization/Bayesian_Optimization_Case_Study/Bayesian Optimization Case Study/flight_delays_test.csv.zip'

In [14]:
# read_csv train and test data
train_df = pd.read_csv(data_train_path)
test_df = pd.read_csv(data_test_path)

**<font color='teal'> Print the top five rows of the train dataframe and review the columns in the data. </font>**

In [15]:
# head of train_df
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


**<font color='teal'> Use the describe function to review the numeric columns in the train dataframe. </font>**

In [16]:
# describe train_df
train_df.describe()

,DepTime,Distance
count,100000.000000,100000.00000
mean,1341.523880,729.39716
std,476.378445,574.61686
min,1.000000,30.00000
25%,931.000000,317.00000
50%,1330.000000,575.00000
75%,1733.000000,957.00000
max,2534.000000,4962.00000


Observations:
* Notice, `DepTime` is the departure time in a numeric representation in 2400 hours.

 **<font color='teal'>The response variable is 'dep_delayed_15min' which is a categorical column, so we need to map the Y for yes and N for no values to 1 and 0. Run the code in the next cell to do this.</font>**

In [17]:
#train_df = train_df[train_df.DepTime <= 2400].copy()
y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values

In [19]:
## now i will check to see if this worked
y_train[0:10]

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0])

## Feature Engineering
Use these defined functions to create additional features for the model. Run the cell to add the functions to your workspace.

In [20]:
# sklearn labelencoder
def label_enc(df_column):
    df_column = LabelEncoder().fit_transform(df_column)
    return df_column


def make_harmonic_features_sin(value, period=2400):
    value *= 2 * np.pi / period
    return np.sin(value)

def make_harmonic_features_cos(value, period=2400):
    value *= 2 * np.pi / period
    return np.cos(value)

def feature_eng(df):
    df['flight'] = df['Origin']+df['Dest']
    df['Month'] = df.Month.map(lambda x: x.split('-')[-1]).astype('int32')
    df['DayofMonth'] = df.DayofMonth.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['begin_of_month'] = (df['DayofMonth'] < 10).astype('uint8')
    df['midddle_of_month'] = ((df['DayofMonth'] >= 10)&(df['DayofMonth'] < 20)).astype('uint8')
    df['end_of_month'] = (df['DayofMonth'] >= 20).astype('uint8')
    df['DayOfWeek'] = df.DayOfWeek.map(lambda x: x.split('-')[-1]).astype('uint8')
    df['hour'] = df.DepTime.map(lambda x: x/100).astype('int32')
    df['morning'] = df['hour'].map(lambda x: 1 if (x <= 11)& (x >= 7) else 0).astype('uint8')
    df['day'] = df['hour'].map(lambda x: 1 if (x >= 12) & (x <= 18) else 0).astype('uint8')
    df['evening'] = df['hour'].map(lambda x: 1 if (x >= 19) & (x <= 23) else 0).astype('uint8')
    df['night'] = df['hour'].map(lambda x: 1 if (x >= 0) & (x <= 6) else 0).astype('int32')
    df['winter'] = df['Month'].map(lambda x: x in [12, 1, 2]).astype('int32')
    df['spring'] = df['Month'].map(lambda x: x in [3, 4, 5]).astype('int32')
    df['summer'] = df['Month'].map(lambda x: x in [6, 7, 8]).astype('int32')
    df['autumn'] = df['Month'].map(lambda x: x in [9, 10, 11]).astype('int32')
    df['holiday'] = (df['DayOfWeek'] >= 5).astype(int)
    df['weekday'] = (df['DayOfWeek'] < 5).astype(int)
    df['airport_dest_per_month'] = df.groupby(['Dest', 'Month'])['Dest'].transform('count')
    df['airport_origin_per_month'] = df.groupby(['Origin', 'Month'])['Origin'].transform('count')
    df['airport_dest_count'] = df.groupby(['Dest'])['Dest'].transform('count')
    df['airport_origin_count'] = df.groupby(['Origin'])['Origin'].transform('count')
    df['carrier_count'] = df.groupby(['UniqueCarrier'])['Dest'].transform('count')
    df['carrier_count_per month'] = df.groupby(['UniqueCarrier', 'Month'])['Dest'].transform('count')
    df['deptime_cos'] = df['DepTime'].map(make_harmonic_features_cos)
    df['deptime_sin'] = df['DepTime'].map(make_harmonic_features_sin)
    df['flightUC'] = df['flight']+df['UniqueCarrier']
    df['DestUC'] = df['Dest']+df['UniqueCarrier']
    df['OriginUC'] = df['Origin']+df['UniqueCarrier']
    return df.drop('DepTime', axis=1)

Concatenate the training and testing dataframes.


In [21]:
# concat train and test dfs
full_df = pd.concat([train_df.drop('dep_delayed_15min', axis=1), test_df])
full_df = feature_eng(full_df)

In [22]:
## quick look at full_df
full_df.head()

,Month,DayofMonth,DayOfWeek,UniqueCarrier,Origin,Dest,Distance,flight,begin_of_month,midddle_of_month,...,airport_origin_per_month,airport_dest_count,airport_origin_count,carrier_count,carrier_count_per month,deptime_cos,deptime_sin,flightUC,DestUC,OriginUC
0,8,21,7,AA,ATL,DFW,732,ATLDFW,0,0,...,1019,8290,11387,18024,1569,0.343660,-0.939094,ATLDFWAA,DFWAA,ATLAA
1,4,20,3,US,PIT,MCO,834,PITMCO,0,0,...,105,3523,1390,13069,1094,-0.612907,-0.790155,PITMCOUS,MCOUS,PITUS
2,9,2,5,XE,RDU,CLE,416,RDUCLE,1,0,...,136,2247,1747,11737,977,-0.835807,-0.549023,RDUCLEXE,CLEXE,RDUXE
3,11,25,6,OO,DEN,MEM,872,DENMEM,0,0,...,514,1785,6222,15343,1242,-0.884988,0.465615,DENMEMOO,MEMOO,DENOO
4,10,7,6,WN,MDW,OMA,423,MDWOMA,1,0,...,226,687,2571,30958,2674,0.073238,-0.997314,MDWOMAWN,OMAWN,MDWWN


In [23]:
## info of full_df
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200000 entries, 0 to 99999
Data columns (total 33 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Month                     200000 non-null  int32  
 1   DayofMonth                200000 non-null  uint8  
 2   DayOfWeek                 200000 non-null  uint8  
 3   UniqueCarrier             200000 non-null  object 
 4   Origin                    200000 non-null  object 
 5   Dest                      200000 non-null  object 
 6   Distance                  200000 non-null  int64  
 7   flight                    200000 non-null  object 
 8   begin_of_month            200000 non-null  uint8  
 9   midddle_of_month          200000 non-null  uint8  
 10  end_of_month              200000 non-null  uint8  
 11  hour                      200000 non-null  int32  
 12  morning                   200000 non-null  uint8  
 13  day                       200000 non-null  uint8  

Apply the earlier defined feature engineering functions to the full dataframe.

In [24]:
for column in ['UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']:
    full_df[column] = label_enc(full_df[column])


Split the new full dataframe into X_train and X_test.

In [25]:
# split X_train and X_test
X_train = full_df[:train_df.shape[0]]
X_test = full_df[train_df.shape[0]:]

Create a list of the categorical features.

In [26]:
# categorical feature list
categorical_features = ['Month',  'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest','flight',  'flightUC', 'DestUC', 'OriginUC']

Let's build a light GBM model to test the bayesian optimizer.

### [LightGBM](https://lightgbm.readthedocs.io/en/latest/) is a gradient boosting framework that uses tree-based learning algorithms. It is designed to be distributed and efficient with the following advantages:

* Faster training speed and higher efficiency.
* Lower memory usage.
* Better accuracy.
* Support of parallel and GPU learning.
* Capable of handling large-scale data.

First, we define the function we want to maximize and that will count cross-validation metrics of lightGBM for our parameters.

Some params such as num_leaves, max_depth, min_child_samples, min_data_in_leaf should be integers.

In [39]:
# function for lightGBM
def lgb_eval(num_leaves,max_depth,lambda_l2,lambda_l1,min_child_samples, min_data_in_leaf):
    params = {
        "objective" : "binary",
        "metric" : 'auc',
        'is_unbalance': True,
        "num_leaves" : int(num_leaves),
        "max_depth" : int(max_depth),
        "lambda_l2" : lambda_l2,
        "lambda_l1" : lambda_l1,
        "num_threads" : 20,
        "min_child_samples" : int(min_child_samples),
        'min_data_in_leaf': int(min_data_in_leaf),
        "learning_rate" : 0.03,
        "subsample_freq" : 5,
        "bagging_seed" : 42,
        "verbosity" : -1
    }
    lgtrain = lightgbm.Dataset(
        X_train, y_train, categorical_feature=categorical_features
    )
    cb_early_stopping = lightgbm.early_stopping(stopping_rounds=100, verbose=False)


    cv_result = lightgbm.cv(params,
                       lgtrain,
                       1000,
                       stratified=True,
                       nfold=3,
                       callbacks=[cb_early_stopping]
    )
    return cv_result['valid auc-mean'][-1]

Apply the Bayesian optimizer to the function we created in the previous step to identify the best hyperparameters. We will run 10 iterations and set init_points = 2.


In [38]:
# apply BayesianOptimization function to lgb_eval function
lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (25, 4000),
                                                'max_depth': (5, 63),
                                                'lambda_l2': (0.0, 0.05),
                                                'lambda_l1': (0.0, 0.05),
                                                'min_child_samples': (50, 10000),
                                                'min_data_in_leaf': (100, 2000)
                                                })


## print out results
lgbBO.maximize(n_iter=10, init_points=2)

|   iter    |  target   | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_da... | num_le... |
-------------------------------------------------------------------------------------------------
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[42]	cv_agg's valid auc: 0.720182 + 0.00402495
| 1         | 0.7202    | 0.008613  | 0.01286   | 12.76     | 9.238e+03 | 225.1     | 834.8     |
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[26]	cv_agg's valid auc: 0.720214 + 0.0036596
| 2         | 0.7202    | 0.03599   | 0.000307  | 48.54     | 7.508e+03 | 344.9     | 2.112e+03 |
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[189]	cv_agg's valid auc: 0.724956 + 0.00364425
| 3         | 0.725     | 0.03989   | 0.02214   | 20.33     | 3.927e+03 | 625.3     | 80.45     |
Training until validation scores don't improve for 100 rounds
Early stopping, 

 **<font color='teal'> Print the best result by using the '.max' function.</font>**

In [40]:
lgbBO.max

{'target': 0.7436914665535695,
 'params': {'lambda_l1': 0.01707120255891231,
  'lambda_l2': 0.01134735496361799,
  'max_depth': 33.393474067073626,
  'min_child_samples': 5180.8712546976785,
  'min_data_in_leaf': 1795.8389035473076,
  'num_leaves': 3138.1344077563667}}

Review the process at each step by using the '.res[0]' function.

In [41]:
lgbBO.res[0]

{'target': 0.7201816382856626,
 'params': {'lambda_l1': 0.008612781785353163,
  'lambda_l2': 0.012862533141767324,
  'max_depth': 12.759316730196632,
  'min_child_samples': 9238.257657514338,
  'min_data_in_leaf': 225.1030079853561,
  'num_leaves': 834.8017067186553}}

In [50]:
lgbBO.res[1]

{'target': 0.7202143183782398,
 'params': {'lambda_l1': 0.03599313418503853,
  'lambda_l2': 0.00030704383537480395,
  'max_depth': 48.54038903475322,
  'min_child_samples': 7508.317531837112,
  'min_data_in_leaf': 344.9461449341372,
  'num_leaves': 2112.1565379699555}}

# Summary
* The optimal parameters were defined by the model above with the lgbBO.max function. We can then go ahead and train a model with these parameters.